In [ ]:
#installations
#!pip install --quiet SimpleITK
#Anaconda Powershell
#:L
#conda activate data_processing
#jupyter serverextension enable -- py jupyter_http_over_ws
#jupyter notebook --NotebookApp.allow_origin='https://colab.research.google.com' --port=8892 --NotebookApp.port_retries=0

In [ ]:
#imports
import os
import torch
import wandb
import pandas as pd
import torch.nn as nn
from einops import rearrange
import sys
#this path must be specified
sys.path.append('AnishSalvi/ImageRx/PAD-Net/vRAD_Dataset_Inputs/Transformer_Explainability')
from modules.layers_ours import *
from baselines.ViT.helpers import load_pretrained
from baselines.ViT.weight_init import trunc_normal_
from baselines.ViT.layer_helpers import to_2tuple
from baselines.ViT.ViT_explanation_generator import LRP
from PIL import Image
import torchvision.transforms as transforms
#import matplotlib.pyplot as plt
from torch import nn
import sklearn
import numpy as np
import SimpleITK as sitk
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import random
import torchvision
#from livelossplot import PlotLosses
#import scipy
#import sklearn
#from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
#from sklearn.utils import shuffle
#from sklearn.metrics import confusion_matrix
#import seaborn as sns
#from ctviewer import CTViewer
#from sklearn.metrics import RocCurveDisplay
import datetime
from datetime import datetime
import json
#check if the gpu machine is available
if torch.cuda.is_available():
  device = 'cuda'
  gpu = torch.cuda.get_device_name(0)
  print('Device: ', gpu)
else:
  device = 'cpu'
  gpu = None
  print('Device', device)

#wanddb
key = "23f8a4a10d7a355a4469d7d7b6b696b8536cba35"
#Weights and Bias
if key:
  wandb.login(key=key) #API Key is in your wandb account, under settings (wandb.ai/settings)

Device:  NVIDIA GeForce RTX 3060


wandb: Currently logged in as: anish_s. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\MeDCaVE_3/.netrc


In [ ]:
#functions

#functions
def compute_rollout_attention(all_layer_matrices, start_layer=0):
    # adding residual consideration
    num_tokens = all_layer_matrices[0].shape[1]
    batch_size = all_layer_matrices[0].shape[0]
    eye = torch.eye(num_tokens).expand(batch_size, num_tokens, num_tokens).to(all_layer_matrices[0].device)
    all_layer_matrices = [all_layer_matrices[i] + eye for i in range(len(all_layer_matrices))]
    # all_layer_matrices = [all_layer_matrices[i] / all_layer_matrices[i].sum(dim=-1, keepdim=True)
    #                       for i in range(len(all_layer_matrices))]
    joint_attention = all_layer_matrices[start_layer]
    for i in range(start_layer+1, len(all_layer_matrices)):
        joint_attention = all_layer_matrices[i].bmm(joint_attention)
    return joint_attention

class Mlp(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = Linear(in_features, hidden_features)
        self.act = GELU()
        self.fc2 = Linear(hidden_features, out_features)
        self.drop = Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x

    def relprop(self, cam, **kwargs):
        cam = self.drop.relprop(cam, **kwargs)
        cam = self.fc2.relprop(cam, **kwargs)
        cam = self.act.relprop(cam, **kwargs)
        cam = self.fc1.relprop(cam, **kwargs)
        return cam


class Attention(nn.Module):
    def __init__(self, dim, num_heads=8, qkv_bias=False,attn_drop=0., proj_drop=0.):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        # NOTE scale factor was wrong in my original version, can set manually to be compat with prev weights
        self.scale = head_dim ** -0.5

        # A = Q*K^T
        self.matmul1 = einsum('bhid,bhjd->bhij')
        # attn = A*V
        self.matmul2 = einsum('bhij,bhjd->bhid')

        self.qkv = Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = Dropout(attn_drop)
        self.proj = Linear(dim, dim)
        self.proj_drop = Dropout(proj_drop)
        self.softmax = Softmax(dim=-1)

        self.attn_cam = None
        self.attn = None
        self.v = None
        self.v_cam = None
        self.attn_gradients = None

    def get_attn(self):
        return self.attn

    def save_attn(self, attn):
        self.attn = attn

    def save_attn_cam(self, cam):
        self.attn_cam = cam

    def get_attn_cam(self):
        return self.attn_cam

    def get_v(self):
        return self.v

    def save_v(self, v):
        self.v = v

    def save_v_cam(self, cam):
        self.v_cam = cam

    def get_v_cam(self):
        return self.v_cam

    def save_attn_gradients(self, attn_gradients):
        self.attn_gradients = attn_gradients

    def get_attn_gradients(self):
        return self.attn_gradients

    #this may cause issues for 3d? or does it act upoon the patches

    def forward(self, x):
        b, n, _, h = *x.shape, self.num_heads
        qkv = self.qkv(x)
        q, k, v = rearrange(qkv, 'b n (qkv h d) -> qkv b h n d', qkv=3, h=h)

        self.save_v(v)

        dots = self.matmul1([q, k]) * self.scale

        attn = self.softmax(dots)
        attn = self.attn_drop(attn)

        self.save_attn(attn)
        attn.register_hook(self.save_attn_gradients)

        out = self.matmul2([attn, v])
        out = rearrange(out, 'b h n d -> b n (h d)')

        out = self.proj(out)
        out = self.proj_drop(out)
        return out

    def relprop(self, cam, **kwargs):
        cam = self.proj_drop.relprop(cam, **kwargs)
        cam = self.proj.relprop(cam, **kwargs)
        cam = rearrange(cam, 'b n (h d) -> b h n d', h=self.num_heads)

        # attn = A*V
        (cam1, cam_v)= self.matmul2.relprop(cam, **kwargs)
        cam1 /= 2
        cam_v /= 2

        self.save_v_cam(cam_v)
        self.save_attn_cam(cam1)

        cam1 = self.attn_drop.relprop(cam1, **kwargs)
        cam1 = self.softmax.relprop(cam1, **kwargs)

        # A = Q*K^T
        (cam_q, cam_k) = self.matmul1.relprop(cam1, **kwargs)
        cam_q /= 2
        cam_k /= 2

        cam_qkv = rearrange([cam_q, cam_k, cam_v], 'qkv b h n d -> b n (qkv h d)', qkv=3, h=self.num_heads)

        return self.qkv.relprop(cam_qkv, **kwargs)


class Block(nn.Module):

    def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=False, drop=0., attn_drop=0.):
        super().__init__()
        self.norm1 = LayerNorm(dim, eps=1e-6)
        self.attn = Attention(
            dim, num_heads=num_heads, qkv_bias=qkv_bias, attn_drop=attn_drop, proj_drop=drop)
        self.norm2 = LayerNorm(dim, eps=1e-6)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, drop=drop)

        self.add1 = Add()
        self.add2 = Add()
        self.clone1 = Clone()
        self.clone2 = Clone()

    def forward(self, x):
        x1, x2 = self.clone1(x, 2)
        x = self.add1([x1, self.attn(self.norm1(x2))])
        x1, x2 = self.clone2(x, 2)
        x = self.add2([x1, self.mlp(self.norm2(x2))])
        return x

    def relprop(self, cam, **kwargs):
        (cam1, cam2) = self.add2.relprop(cam, **kwargs)
        cam2 = self.mlp.relprop(cam2, **kwargs)
        cam2 = self.norm2.relprop(cam2, **kwargs)
        cam = self.clone2.relprop((cam1, cam2), **kwargs)

        (cam1, cam2) = self.add1.relprop(cam, **kwargs)
        cam2 = self.attn.relprop(cam2, **kwargs)
        cam2 = self.norm1.relprop(cam2, **kwargs)
        cam = self.clone1.relprop((cam1, cam2), **kwargs)
        return cam

#3D patch embedder
class PatchEmbed3D(nn.Module):
  """ Image to Patch Embedding
  """
  def __init__(self, img_size = (200, 100, 150), patch_size = (20, 10, 15), in_chans = 1, embed_dim = 768):
    super().__init__()
    img_size = to_2tuple(img_size)
    patch_size = to_2tuple(patch_size)
    num_patches = (img_size[2] // patch_size[2]) * (img_size[1] // patch_size[1]) * (img_size[0] // patch_size[0])
    self.img_size = img_size
    self.patch_size = patch_size
    self.num_patches = num_patches
    self.proj = torch.nn.Conv3d(in_chans, embed_dim, kernel_size = patch_size, stride = patch_size)

  def forward(self, x):
    B, C, H, W, D = x.shape
    # FIXME look at relaxing size constraints
    assert H == self.img_size[0] and W == self.img_size[1] and D == self.img_size[2], \
      f"Input image size ({H}*{W}*{D}) doesn't match model ({self.img_size[0]} * {self.img_size[1]} * {self.img_size[2]})."
    x = self.proj(x).flatten(2).transpose(1, 2)
    return x

  def relprop(self, cam, **kwargs):
    cam = cam.transpose(1,2) #why transpose?
    cam = cam.reshape(cam.shape[0], cam.shape[1], cam.reshape[2], #is the cam in 3d as well?
                      (self.img_size[0] // self.patch_size[0]),
                      (self.img_size[1] // self.patch_size[1]),
                      (self.img_size[2] // self.patch_size[2])
                      )
    return self.proj.relprop(cam, **kwargs)


class VisionTransformer(nn.Module):
    """ Vision Transformer with support for patch or hybrid CNN input stage
    """
    def __init__(self, img_size=(200, 100, 150), patch_size=(20, 10, 15), in_chans=1, num_classes=2, embed_dim=768, depth=12,
                 num_heads=12, mlp_ratio=4., qkv_bias=False, mlp_head=False, drop_rate=0., attn_drop_rate=0.):
        super().__init__()
        self.num_classes = num_classes
        self.num_features = self.embed_dim = embed_dim  # num_features for consistency with other models
        self.patch_embed = PatchEmbed3D(
                img_size=img_size, patch_size=patch_size, in_chans=in_chans, embed_dim=embed_dim)
        num_patches = self.patch_embed.num_patches

        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches + 1, embed_dim))
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))

        self.blocks = nn.ModuleList([
            Block(
                dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias,
                drop=drop_rate, attn_drop=attn_drop_rate)
            for i in range(depth)])

        self.norm = LayerNorm(embed_dim)
        if mlp_head:
            # paper diagram suggests 'MLP head', but results in 4M extra parameters vs paper
            self.head = Mlp(embed_dim, int(embed_dim * mlp_ratio), num_classes)
        else:
            # with a single Linear layer as head, the param count within rounding of paper
            self.head = Linear(embed_dim, num_classes)

        # FIXME not quite sure what the proper weight init is supposed to be,
        # normal / trunc normal w/ std == .02 similar to other Bert like transformers
        trunc_normal_(self.pos_embed, std=.02)  # embeddings same as weights?
        trunc_normal_(self.cls_token, std=.02)
        self.apply(self._init_weights)

        self.pool = IndexSelect()
        self.add = Add()

        self.inp_grad = None

    def save_inp_grad(self,grad):
        self.inp_grad = grad

    def get_inp_grad(self):
        return self.inp_grad


    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    @property
    def no_weight_decay(self):
        return {'pos_embed', 'cls_token'}

    def forward(self, x):
        B = x.shape[0]
        x = self.patch_embed(x)

        cls_tokens = self.cls_token.expand(B, -1, -1)  # stole cls_tokens impl from Phil Wang, thanks
        x = torch.cat((cls_tokens, x), dim=1)
        x = self.add([x, self.pos_embed])

        x.register_hook(self.save_inp_grad)

        for blk in self.blocks:
            x = blk(x)

        x = self.norm(x)
        x = self.pool(x, dim=1, indices=torch.tensor(0, device=x.device))
        x = x.squeeze(1)
        x = self.head(x)
        return x

    def relprop(self, cam=None,method="transformer_attribution", is_ablation=False, start_layer=0, **kwargs):
        # print(kwargs)
        # print("conservation 1", cam.sum())
        cam = self.head.relprop(cam, **kwargs)
        cam = cam.unsqueeze(1)
        cam = self.pool.relprop(cam, **kwargs)
        cam = self.norm.relprop(cam, **kwargs)
        for blk in reversed(self.blocks):
            cam = blk.relprop(cam, **kwargs)

        # print("conservation 2", cam.sum())
        # print("min", cam.min())

        if method == "full":
            (cam, _) = self.add.relprop(cam, **kwargs)
            cam = cam[:, 1:]
            cam = self.patch_embed.relprop(cam, **kwargs)
            # sum on channels
            cam = cam.sum(dim=1)
            return cam

        elif method == "rollout":
            # cam rollout
            attn_cams = []
            for blk in self.blocks:
                attn_heads = blk.attn.get_attn_cam().clamp(min=0)
                avg_heads = (attn_heads.sum(dim=1) / attn_heads.shape[1]).detach()
                attn_cams.append(avg_heads)
            cam = compute_rollout_attention(attn_cams, start_layer=start_layer)
            cam = cam[:, 0, 1:]
            return cam

        # our method, method name grad is legacy
        elif method == "transformer_attribution" or method == "grad":
            cams = []
            for blk in self.blocks:
                grad = blk.attn.get_attn_gradients()
                cam = blk.attn.get_attn_cam()
                cam = cam[0].reshape(-1, cam.shape[-1], cam.shape[-1])
                grad = grad[0].reshape(-1, grad.shape[-1], grad.shape[-1])
                cam = grad * cam
                cam = cam.clamp(min=0).mean(dim=0)
                cams.append(cam.unsqueeze(0))
            rollout = compute_rollout_attention(cams, start_layer=start_layer)
            cam = rollout[:, 0, 1:]
            return cam

        elif method == "last_layer":
            cam = self.blocks[-1].attn.get_attn_cam()
            cam = cam[0].reshape(-1, cam.shape[-1], cam.shape[-1])
            if is_ablation:
                grad = self.blocks[-1].attn.get_attn_gradients()
                grad = grad[0].reshape(-1, grad.shape[-1], grad.shape[-1])
                cam = grad * cam
            cam = cam.clamp(min=0).mean(dim=0)
            cam = cam[0, 1:]
            return cam

        elif method == "last_layer_attn":
            cam = self.blocks[-1].attn.get_attn()
            cam = cam[0].reshape(-1, cam.shape[-1], cam.shape[-1])
            cam = cam.clamp(min=0).mean(dim=0)
            cam = cam[0, 1:]
            return cam

        elif method == "second_layer":
            cam = self.blocks[1].attn.get_attn_cam()
            cam = cam[0].reshape(-1, cam.shape[-1], cam.shape[-1])
            if is_ablation:
                grad = self.blocks[1].attn.get_attn_gradients()
                grad = grad[0].reshape(-1, grad.shape[-1], grad.shape[-1])
                cam = grad * cam
            cam = cam.clamp(min=0).mean(dim=0)
            cam = cam[0, 1:]
            return cam


def _conv_filter(state_dict, patch_size=30):
    """ convert patch embedding weight from manual patchify + linear proj to conv"""
    out_dict = {}
    for k, v in state_dict.items():
        if 'patch_embed.proj.weight' in k:
            v = v.reshape((v.shape[0], 3, patch_size, patch_size))
        out_dict[k] = v
    return out_dict

#Dataset
class CustomImageDataset(Dataset):
    def __init__(self, df, col_image, col_label, aug = False, shuffle = False):
      #params
      self.df = df
      if shuffle:
        self.df = self.df.sample(frac = 1, random_state = 42,).reset_index()
      self.aug = aug
      self.col_image = col_image
      self.col_label = col_label

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
      #row
      row = self.df.iloc[idx]

      #read image
      image = torch.Tensor(np.expand_dims(sitk.GetArrayFromImage(sitk.ReadImage(row[self.col_image])), axis = 0))
      label = row[self.col_label]

      #if aug
      if self.aug:
        #augmentation (increase this given size of the data)
        if random.random() > 0.5: #0.7 0.8
          #horizontal
          image = torchvision.transforms.functional.hflip(image)
        if random.random() > 0.5:
          #vertical
          image = torchvision.transforms.functional.vflip(image)
        #if random.random() > 0.5:
          #rotate
          image = torchvision.transforms.functional.rotate(image,
                                                           random.choice([30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330]))
        #noise
        #if random.random() > 0.5:
          #noise = torch.normal(random.uniform(-0.9, 0.9), random.uniform(0.01, 0.1), image.shape)
          #image = image + noise

        #alternate roations doesn't work due to shape
        #if random.random() > 0.5:
        #  ls = random.sample([1,2,3], 2)
        #  k = random.randint(1, 3)
        #  image = torch.rot90(image, k, ls)

        #affine or perspective may be more aggressive? tranlate and scale --> have to make sure are valid inputs!
        #hopefully increase scale invariance!
        #if random.random() > 0.5:
          #careful not to cut off
          #x = random.choice([5, 10, 15, -5, -10, -15])
          #y = random.choice([5, 10, 15, -5, -10, -15])
          #higher indicates more zoom in (AAA usually in center; may decide to remove if model not fitting correctly)
          #may also remove the perspective aspect of the problem (gauging if bigger or smaller!)
          #scaling = random.choice([0.9, 1.1, 1.5, 1.9])
          #image = torchvision.transforms.functional.affine(image, angle = 0, translate = (x, y), scale = scaling, shear = 0, fill = -1,
          #interpolation = torchvision.transforms.InterpolationMode.BILINEAR)
      #return
      return image, label

#save json file
def save_params(hyper_params, save_path):
  json_string = json.dumps(hyper_params)
  with open(save_path, 'w') as outfile:
    outfile.write(json_string)

#load the train params back in
def load_params(fpath):
  # Opening JSON file
  with open(fpath) as json_file:
    data = json.load(json_file)
  return data

#get the optimizer
def get_optimizer(model_config, model):
  #AdamW
  if model_config['optimizer'] == 'AdamW':
    optimizer = torch.optim.AdamW(model.parameters(), lr = model_config['init_lr'], weight_decay = model_config['weight_decay'])
  #Adam
  if model_config['optimizer'] == 'Adam':
    optimizer = torch.optim.Adam(model.parameters(), lr = model_config['init_lr'], weight_decay = model_config['weight_decay'])
  #NAdam
  if model_config['optimizer'] == 'NAdam':
    optimizer = torch.optim.NAdam(model.parameters(), lr = model_config['init_lr'], weight_decay = model_config['weight_decay'])
  #RAdam
  if model_config['optimizer'] == 'RAdam':
    optimizer = torch.optim.RAdam(model.parameters(), lr = model_config['init_lr'], weight_decay = model_config['weight_decay'])
  #return
  return optimizer

#get the model
def get_model(model_config):
  #model
  model = VisionTransformer(
      img_size = model_config['img_size'],
      patch_size = model_config['patch_size'],
      in_chans = model_config['in_chans'],
      num_classes = model_config['num_classes'],
      embed_dim = model_config['embed_dim'],
      depth = model_config['depth'],
      num_heads = model_config['num_heads'],
      mlp_ratio = model_config['mlp_ratio'],
      qkv_bias = model_config['qkv_bias'],
      mlp_head = model_config['mlp_head'],
      drop_rate = model_config['drop_rate'],
      attn_drop_rate = model_config['attn_drop_rate']
  )
  #return
  return model

#model saving policy
def save_model(model_config, model):
  #save model use weights instead
  model.eval()
  #depending on choice
  if model_config['save_weights_only']:
    torch.save(model.state_dict(), model_config['save_folder'] + 'model_weights.pth')
  else:
    torch.save(model, model_config['save_folder'] + 'model.pth')
  #save info
  save_params(model_config, model_config['save_folder'] + 'model_config.json')

#update previously saved config only
def update_config_stopearly(save_path):
  #load
  model_config = load_params(save_path + 'model_config.json')
  #update
  model_config['early_stopping']['stopped_early'] = True
  #save
  save_params(model_config, save_path + 'model_config.json')

#new saving policy
def new_saving_policy(early_stop, best_model, model_config, model, epoch):
  #if there is an early stop
  if early_stop:
    #exit training
    exit_training = True
    #has the model been already saved?
    if model_config['early_stopping']['model_criteria']:
      #save just the config with update
      update_config_stopearly(model_config['save_folder'])
    #if not already saved
    else:
      #update
      model_config['early_stopping']['stopped_early'] = True
      #save the model and config
      save_model(model_config, model)
  #if there is not early stop
  else:
    #exit
    exit_training = False
    #need to log that we did not exit training early
    model_config['early_stopping']['stopped_early'] = False
    #you want to save the model every n_epochs often
    if model_config['save_best_model'] == False:
      #check if epocch is divisible and nonzero
      if (epoch % model_config['save_after_n_epochs'] == 0) and (epoch != 0):
        #indicate the model was saved
        model_config['early_stopping']['model_criteria'] = True
        #save
        save_model(model_config, model)
    else:
      #you want to save the best model
      if model_config['epochs_trained'] >= model_config['save_after_n_epochs']:
        #check if current model is the best model
        if best_model:
          #log that it is the best model
          model_config['early_stopping']['best_model'] = True
          #indicate the model was saved
          model_config['early_stopping']['model_criteria'] = True
          #then save
          save_model(model_config, model)
        #if current model is not the best model but want to save for the initital run
        if (best_model == False) and (model_config['epochs_trained'] == model_config['save_after_n_epochs']):
          #inidicate the model was saved
          model_config['early_stopping']['model_criteria'] = True
          #save the model and config
          save_model(model_config, model)

  #return
  return model_config, exit_training

#class earlystopping
class EarlyStopping:
  #early stop if validation does not improve for given patience
  def __init__(self, model_config, verbose = True, trace_func = print):
    #set up
    self.patience = model_config['early_stopping']['patience']
    self.delta = model_config['early_stopping']['delta']
    self.verbose = verbose
    self.trace_func = trace_func
    self.counter = 0
    self.best_score = None
    self.best_model = False
    self.early_stop = False

  #call
  def __call__(self, val_loss):
    #neg val loss
    score = -val_loss
    #init condition
    if self.best_score is None:
      self.best_score = score
    #count number of times model failed to meet the condition
    elif score < self.best_score + self.delta:
      self.counter += 1
      self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
      self.best_model = False
      #identify when early stopping is required
      if self.counter >= self.patience:
        self.early_stop = True
    #if the model shows best score
    else:
      #get the score and counter
      self.best_score = score
      self.counter = 0
      self.best_model = True
    #return the interl
    return self.early_stop, self.best_model

#get the loss fn
def get_loss(model_config, device, df = None):
  #criterion if manually weighted
  if (model_config['loss'] == 'CE') and (model_config['weighted'] == True):
    #get
    criterion = nn.CrossEntropyLoss(weight = torch.Tensor(model_config['weights']).to(device))
  #criterion if not weighted
  if (model_config['loss'] == 'CE') and (model_config['weighted'] == False):
    #get
    criterion = nn.CrossEntropyLoss()
  #criterion if using automatic weighting?
  if (model_config['loss'] == 'CE') and (model_config['weighted'] == 'auto'):
    #get
    criterion = nn.CrossEntropyLoss(weight = torch.Tensor(auto_weights(model_config, df)).to(device))
  #return
  return criterion

#get scheduler
def get_scheduler(model_config, optimizer):
  #plateau
  if model_config['scheduler']['description'] == 'plateau':
    #get the scheduler
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode = model_config['scheduler']['mode'],
                                                           factor = model_config['scheduler']['factor'],
                                                           patience = model_config['scheduler']['patience'],
                                                           threshold = model_config['scheduler']['threshold'],
                                                           threshold_mode = model_config['scheduler']['threshold_mode'],
                                                           cooldown = model_config['scheduler']['cooldown'],
                                                           min_lr = model_config['scheduler']['min_lr'],
                                                           eps = model_config['scheduler']['eps'],
                                                           verbose = True)
  #return
  return scheduler

#get sampler
def get_sampler(df, model_config):
  labels_unique, counts = np.unique(df[model_config['col_label']], return_counts = True)
  class_weights = [sum(counts) / c for c in counts]
  example_weights = [class_weights[e] for e in df[model_config['col_label']]]
  sampler = torch.utils.data.WeightedRandomSampler(example_weights, len(df['Annotation_Label']))
  return sampler

#get loss weights automatically
def auto_weights(model_config, df):
  #get class weights
  y = df[model_config['col_label']].to_numpy().astype(np.int8)
  #get the class weights
  class_weights = sklearn.utils.class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y)
  #convert to tensor
  class_weights = torch.tensor(class_weights, dtype = torch.float)
  #return (may have to send to gpu)
  return class_weights

In [ ]:
#parameters in wandb format
sweep_config = {
    #name decided later (sweep name)
    'name': None,
    #sweep method
    'method': 'grid',
    #metric
    'metric': {
        'name': 'val_acc',
        'goal': 'maximize',
    },
    #values which may be altered wandb wants all components
    'parameters': {
        #description
        'description': {'value': '3D ViT for classification of vRAD data for 2 folds'},
        #project
        'project':{'value': '3D-PAD-ViT-Sweep'},
        'model': {'value': None}, #placeholder for actual name
        #documentation
        'data_path': {'value': 'AnishSalvi/ImageRx/PAD-Net/ViT_classification/dataset/dataset-2023-03-04-01-37-09.pkl'},
        #path to save the results
        'save_folder': {'value': 'AnishSalvi/ImageRx/PAD-Net/ViT_classification/results/'},
        'col_image': {'value': 'Norm-256-256-256'},
        'col_label': {'value': 'Annotation_Label'},
        'device': {'value': device},
        'val_kfolds': {'value': [4, 5]},
        #model params
        'img_size': {'values': [(256, 256, 256)]},
        'patch_size': {'values': [(64, 64, 64)]},
        'in_chans': {'values': [1]},
        'num_classes': {'values': [3]},
        'embed_dim': {'values': [512]},
        'depth': {'values': [8]},
        'num_heads': {'values': [8]},
        'mlp_ratio': {'values': [8.]},
        'qkv_bias': {'values': [False]},
        'mlp_head': {'values': [True]},
        'drop_rate': {'values': [10e-2]},
        'attn_drop_rate': {'values': [10e-2]},
        #training params
        'aug': {'values': [True]},
        'batch_size': {'values': [4]},
        'init_lr': {'values': [1e-6]},
        'epochs': {'values': [100]},
        'epochs_trained': {'value': 0}, #this is updated in the script!
        'save_after_n_epochs': {'value': 5}, #depends on if you want to save the best model
        'weight_decay': {'values': [1e-4]},
        'optimizer': {'values': ['AdamW']},
        'scheduler': {'values': [
            {'description': 'plateau',
             'mode': 'min',
             'factor': 5e-1,
             'patience': 10,
             'threshold': 1e-3,
             'threshold_mode': 'rel',
             'cooldown': 0,
             'min_lr': 0,
             'eps': 1}]},
        'loss': {'values': ['CE']},
        'weighted': {'values': ['auto']},
        'weights': {'values': [(0.3, 0.5, 0.2)]}, #by default // depends on weighted
        'sampler': {'values':[False]},
        #saving
        'save_weights_only': {'value': True},
        'save_best_model': {'value': True},
        #early stopping
        'early_stopping': {'value':
            {'patience': 10,
            'delta': 1e-4,
            'stopped_early': None,
            'best_model': None,
            'model_criteria': False}
        },
        #log the model loss and acc
        'log_train_loss': {'value': None},
        'log_train_acc': {'value': None},
        'log_val_loss': {'value': None},
        'log_val_acc': {'value': None},
        #updating performance in WandB
        'train_loss': {'value': None},
        'train_acc': {'value': None},
        'val_loss': {'value': None},
        'val_acc': {'value': None}
    }
}


In [ ]:
#main script
def main(config = None):
  #clear workspace when finished with a single model run
  model, x, y_true, y_pred, loss = (None, None, None, None, None)
  dset_train, train_loader, dset_val, val_loader = (None, None, None, None)
  criterion, optimizer, scheduler, early_stopper = (None, None, None, None)
  #reset
  if device == 'cuda':
    torch.cuda.empty_cache()

  #name the model
  model_name = datetime.now().strftime('3D-ViT-classification-%Y-%m-%d-%H-%M-%S')

  #init a new wandb run (config = sweep_config)
  with wandb.init(config = config, name = model_name):
    #set up the config (WandB, locked)
    config = wandb.config
    #dict (not locked)
    model_config = dict(config)
    #name the model
    model_config['model'] = model_name
    #save location
    model_config['save_folder'] = model_config['save_folder'] +  model_config['model'] + '/'
    #create the model folder
    if os.path.isdir(model_config['save_folder']) == False:
      os.mkdir(model_config['save_folder'])

    #training data(not in valkfolds!)
    df_train = df[~df['KFold'].isin(model_config['val_kfolds'])]

    #get the training data (remove the folds corresponding to validation)
    dset_train = CustomImageDataset(df_train,
                                    col_image = model_config['col_image'], col_label = model_config['col_label'],
                                    aug = model_config['aug'], shuffle = False)

    #if train sampler
    if model_config['sampler']:
      train_sampler = get_sampler(df_train, model_config)
    else:
      train_sampler = None

    #train loader
    train_loader = DataLoader(dset_train, sampler = train_sampler, batch_size = model_config['batch_size'])

    #val data (in valkfolds)
    df_val = df[df['KFold'].isin(model_config['val_kfolds'])]

    #get the validation data
    dset_val = CustomImageDataset(df_val,
                                  col_image = model_config['col_image'], col_label = model_config['col_label'],
                                  aug = False, shuffle = False)

    #if val sampler
    if model_config['sampler']:
      val_sampler = get_sampler(df_val, model_config)
    else:
      val_sampler = None

    #val loader
    val_loader = DataLoader(dset_val, sampler = val_sampler, batch_size = model_config['batch_size'])

    #init the model
    model = get_model(model_config)
    #send
    model.to(device)
    #criterion (can get loss based on training data)
    criterion = get_loss(model_config, device, df_train)
    #optimizer
    optimizer = get_optimizer(model_config, model)
    #scheduler
    scheduler = get_scheduler(model_config, optimizer)
    #early stopping (save time during the sweep)
    early_stopper = EarlyStopping(model_config)

    #track in Jupter Notebook
    #liveloss = PlotLosses()
    #logs
    #logs = {}

    #track for later
    log_train_loss = []
    log_train_acc = []
    log_val_loss = []
    log_val_acc = []


    #iterate through the entire dataset
    #+1 for shifting (python starts at 0)
    for epoch in range(model_config['epochs_trained'] + 1, model_config['epochs'] + 1):
      #determine train losses
      train_epoch_loss = 0
      #set for training
      model.train()
      #iterate through the training data
      for i, (x, y_true) in enumerate(train_loader):
        #zero optimizer
        optimizer.zero_grad()
        #send to device
        #x = x.to(device)
        #y_true = y_true.to(device)
        #predict
        y_pred = model(x.to(device))
        #determine loss (should already be averaged)
        loss = criterion(y_pred, y_true.to(device))
        #backward
        loss.backward()
        #step
        optimizer.step()
        #track the loss
        train_epoch_loss = train_epoch_loss + loss.item()
        #reset
        x, y_true, y_pred, loss = (None, None, None, None)
        if device == 'cuda':
          torch.cuda.empty_cache()
      #calculate train loss
      train_loss = train_epoch_loss / len(train_loader)
      #calculate train acc
      train_acc = 1 - train_loss

      #determine validation losses
      val_epoch_loss = 0
      #specify eval
      model.eval()
      #set
      #with torch.no_grad():
      for i, (x, y_true) in enumerate(val_loader):
        #send to device
        #x = x.to(device)
        #y_true = y_true.to(device)
        #predict
        y_pred = model(x.to(device))
        #determine loss
        loss = criterion(y_pred, y_true.to(device))
        #track the loss (shoudld already be averaged)
        val_epoch_loss = val_epoch_loss + loss.item()
        #reset
        x, y_true, y_pred, loss = (None, None, None, None)
        if device == 'cuda':
          torch.cuda.empty_cache()
      #calulate val loss
      val_loss = val_epoch_loss / len(val_loader)
      #calulate val acc
      val_acc = 1 - val_loss

      #scheduler
      scheduler.step(train_loss)

      #record for training
      log_train_loss.append(train_loss)
      log_train_acc.append(train_acc)
      #record for validation
      log_val_loss.append(val_loss)
      log_val_acc.append(val_acc)

      #wont log lossess or acc after early stopping or save best model

      #log the most recent info
      model_config['train_loss'] = train_loss
      model_config['train_acc'] = train_acc
      model_config['val_loss'] = val_loss
      model_config['val_acc'] = val_acc

      #log all the info
      model_config['log_train_loss'] = log_train_loss
      model_config['log_train_acc'] = log_train_acc
      model_config['log_val_loss'] = log_val_loss
      model_config['log_val_acc'] = log_val_acc

      #keep track of each epoch
      model_config['epochs_trained'] = epoch

      #print
      print('Epoch {0} of {1}: Train Loss {2:.2g} & Acc {3:.2g} v Val Loss {4:.2g} and Acc {5:.2g}'.format(epoch, model_config['epochs'],
                                                                                                           train_loss, train_acc, val_loss, val_acc))

      #wandb
      wandb.log(model_config)


      #determine if early stopping is required by validation loss
      early_stop, best_model = early_stopper(val_loss)
      #saving policy and determine if training should be exited based on early stop and best model
      model_config, exit_training = new_saving_policy(early_stop, best_model, model_config, model, epoch)

      #specify the logs
      #prefix = ''
      #logs['Loss'] = train_loss
      #logs['Acc'] = train_acc
      #logs
      #prefix = 'val_'
      #logs[prefix + 'Loss'] = val_loss
      #logs[prefix + 'Acc'] = val_acc

      #living loss
      #liveloss.update(logs)
      #send
      #liveloss.send()

      #exit training early
      if exit_training:
        print('Early Stop: Exit Training')
        break

    #clear workspace when finished with a single model run
    model, x, y_true, y_pred, loss = (None, None, None, None, None)
    dset_train, train_loader, dset_val, val_loader = (None, None, None, None)
    criterion, optimizer, scheduler, early_stopper = (None, None, None, None)
    #reset
    if device == 'cuda':
      torch.cuda.empty_cache()

In [ ]:
#main script

if __name__ == '__main__':

  #read the pickle file
  df = pd.read_pickle(sweep_config['parameters']['data_path']['value'])

  #specify the sweep save location
  sweep_config['name'] = datetime.now().strftime('3D-ViT-sweep-%Y-%m-%d-%H-%M-%S')
  #set
  sweep_config['parameters']['save_folder']['value'] = sweep_config['parameters']['save_folder']['value'] + sweep_config['name'] + '/'
  #create the sweep folder
  if os.path.isdir(sweep_config['parameters']['save_folder']['value']) == False:
    os.mkdir(sweep_config['parameters']['save_folder']['value'])
  #save the sweep config in the sweep folder
  save_params(sweep_config, sweep_config['parameters']['save_folder']['value'] + 'sweep_config.json')
  #now run the main script

  #select the project folder
  sweep_id = wandb.sweep(sweep_config, project = sweep_config['parameters']['project']['value'])
  #execute the search
  wandb.agent(sweep_id, main)
  #finish
  wandb.finish()


Create sweep with ID: pokwf2wx
Sweep URL: https://wandb.ai/anish_s/3D-PAD-ViT-Sweep/sweeps/pokwf2wx


wandb: Agent Starting Run: h01n4yxk with config:
wandb: 	attn_drop_rate: 0.1
wandb: 	aug: True
wandb: 	batch_size: 4
wandb: 	col_image: Norm-256-256-256
wandb: 	col_label: Annotation_Label
wandb: 	data_path: AnishSalvi/ImageRx/PAD-Net/ViT_classification/dataset/dataset-2023-03-04-01-37-09.pkl
wandb: 	depth: 8
wandb: 	description: 3D ViT for classification of vRAD data for 2 folds
wandb: 	device: cuda
wandb: 	drop_rate: 0.1
wandb: 	early_stopping: {'best_model': None, 'delta': 0.0001, 'model_criteria': False, 'patience': 10, 'stopped_early': None}
wandb: 	embed_dim: 512
wandb: 	epochs: 100
wandb: 	epochs_trained: 0
wandb: 	img_size: [256, 256, 256]
wandb: 	in_chans: 1
wandb: 	init_lr: 1e-06
wandb: 	log_train_acc: None
wandb: 	log_train_loss: None
wandb: 	log_val_acc: None
wandb: 	log_val_loss: None
wandb: 	loss: CE
wandb: 	mlp_head: True
wandb: 	mlp_ratio: 8
wandb: 	model: None
wandb: 	num_classes: 3
wandb: 	num_heads: 8
wandb: 	optimizer: AdamW
wandb: 	patch_size: [64, 64, 64]
wandb: 	

Epoch 1 of 100: Train Loss 1.2 & Acc -0.16 v Val Loss 1.2 and Acc -0.2
Epoch 2 of 100: Train Loss 1.1 & Acc -0.053 v Val Loss 1.2 and Acc -0.22
EarlyStopping counter: 1 out of 10
Epoch 3 of 100: Train Loss 1 & Acc 0.0015 v Val Loss 1.2 and Acc -0.19
Epoch 4 of 100: Train Loss 1 & Acc -0.04 v Val Loss 1.3 and Acc -0.26
EarlyStopping counter: 1 out of 10
Epoch 5 of 100: Train Loss 1 & Acc -0.024 v Val Loss 1.2 and Acc -0.21
EarlyStopping counter: 2 out of 10
Epoch 6 of 100: Train Loss 1 & Acc -0.012 v Val Loss 1.2 and Acc -0.23
EarlyStopping counter: 3 out of 10
Epoch 7 of 100: Train Loss 1 & Acc -0.022 v Val Loss 1.2 and Acc -0.25
EarlyStopping counter: 4 out of 10
Epoch 8 of 100: Train Loss 0.95 & Acc 0.049 v Val Loss 1.3 and Acc -0.27
EarlyStopping counter: 5 out of 10
Epoch 9 of 100: Train Loss 0.99 & Acc 0.0096 v Val Loss 1.3 and Acc -0.26
EarlyStopping counter: 6 out of 10
Epoch 10 of 100: Train Loss 0.97 & Acc 0.03 v Val Loss 1.2 and Acc -0.24
EarlyStopping counter: 7 out of 10
Ep

attn_drop_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
aug,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch_size,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
depth,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
drop_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
embed_dim,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epochs,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epochs_trained,▁▁▂▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇▇██
in_chans,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
init_lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mlp_head,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


In [ ]:
#wandb files
import shutil
shutil.rmtree('wandb')
#https://stackoverflow.com/questions/70508960/how-to-free-gpu-memory-in-pytorch

PermissionError: ignored